In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from datetime import datetime

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# diese im Working Directory haben:
import os
path = '/content/drive/MyDrive/techlabs/Github/Daten/' # ggf. anpassen

stammdaten_tree = ET.parse(path + 'MDB_STAMMDATEN.XML')
root = stammdaten_tree.getroot()
#root = ET.parse('/content/drive/MyDrive/techlabs_Daten/MDB_STAMMDATEN.XML').getroot()

zusatzdaten = pd.read_csv(path + "Stammdaten_nachkodiert.csv", sep = ";")
#zusatzdaten = pd.read_csv("/content/drive/MyDrive/techlabs_Daten/Stammdaten_nachkodiert.csv", sep = ";")


df = pd.read_parquet(path + "datatechlabs_2020.parquet")
#df = pd.read_parquet('/content/drive/MyDrive/techlabs_Daten/datatechlabs.parquet')



In [ ]:
#df.head()

In [ ]:
#df.describe()

"Search Party"-Funktionen:
(1) search_party: Findet die aktuelle Partei auf Basis des Datums in den Stammdaten.
(2) search_basic_party: Findet die Partei in den biografischen Angaben in den Stammdaten.
(3) search_lost_party: Findet die Partei in der Liste der händisch nach-kodierten Personen.

In [ ]:
#zusatzdaten.head(20)


Zunächst alle Reden von Politiker*innen, die nicht auf Bundesebene agieren, ausschließen (Bürgermeister*innen, Minister*innen, ...)
--> diese sind Gottseidank die einzigen mit Klammern in der Rollenbezeichnung,
z. B. Ministerpräsident (Brandenburg)
An der Zahl sind es nur 92 :)

In [ ]:
df = df[~df['Rolle'].str.contains(r"\(.*\)")]

In [ ]:
def search_party(root, id, wp, datum, nachname):
  party = "keine"
  for i in root:
    for j in i.findall("ID"):
      if str(id) == j.text:

        stamm_nachname = i.find('NAMEN').find('NAME').find('NACHNAME').text

        for k in i.find("WAHLPERIODEN").findall("WAHLPERIODE"):
          if str(wp) == k.find("WP").text:
            for l in k.find("INSTITUTIONEN"):
              if l.find("INSART_LANG").text == "Fraktion/Gruppe":
                if (l.find("MDBINS_VON").text is None) & (l.find("MDBINS_BIS").text is None):
                  party = l.find("INS_LANG").text
                elif l.find("MDBINS_BIS").text is None:
                  if datetime.strptime(str(datum), "%d.%m.%Y") > datetime.strptime(l.find("MDBINS_VON").text, "%d.%m.%Y"):
                    party = l.find("INS_LANG").text
                elif l.find("MDBINS_VON").text is None:
                  if datetime.strptime(str(datum), "%d.%m.%Y") < datetime.strptime(l.find("MDBINS_BIS").text, "%d.%m.%Y"):
                    party = l.find("INS_LANG").text
                elif (datetime.strptime(l.find("MDBINS_VON").text, "%d.%m.%Y") < datetime.strptime(str(datum), "%d.%m.%Y") < datetime.strptime(l.find("MDBINS_BIS").text, "%d.%m.%Y")):
                    party = l.find("INS_LANG").text

          # Anpassungen auf Basis des Namens:
          if stamm_nachname != nachname:
           if nachname == 'Schäuble' or nachname == 'Steinmeier':   # hier uneindeutig (ID vs. Name)
             party = "Fehler: Name/ Fraktion"

  return party

In [ ]:
#search_party(root = root, id = 11004752, wp = 20, datum = "20.12.2024")

TypeError: search_party() missing 1 required positional argument: 'nachname'

In [ ]:
def search_lost_party(zuordnung, id, nachname):
  for i in range(0, len(zuordnung)):
    if (zuordnung.iloc[i]['ID'] == id):
      if (zuordnung.iloc[i]["Nachname"] == nachname):
        return zuordnung.iloc[i]['Partei']
      else:
        return "NACHNAMENFEHLER"
  return "keine"

In [ ]:
for index, row in df.iterrows():
  if df.loc[index, 'Fraktion'] == 'keine':
    df.loc[index, 'Fraktion'] = search_party(root = root, id = row['Redner*in-ID'], wp = row['Wahlperiode'], datum = row['Datum'], nachname = row['Redner*in Nachname'])
  if df.loc[index, 'Fraktion'] == 'keine':
    df.loc[index, 'Fraktion'] = search_lost_party(zuordnung = zusatzdaten, id = row['Redner*in-ID'], nachname = row["Redner*in Nachname"])

In auskommentiert hier noch die etwas schnellere Variante ohne überprüfung des Nachnamens in zweiter Version (kann eig ignoriert werden)

In [ ]:
#def search_lost_party(zuordnung, id):
#  for i in range(0, len(zuordnung)):
#    if (zuordnung.iloc[i]['ID'] == id):
#      return zuordnung.iloc[i]['Partei']
#  return "keine"

In [ ]:
#for index, row in df.iterrows():
#  if df.loc[index, 'Fraktion'] == 'keine':
#    df.loc[index, 'Fraktion'] = search_party(root = root, id = row['Redner*in-ID'], wp = row['Wahlperiode'], datum = row['Datum'], nachname = row['Redner*in Nachname'])
#  if df.loc[index, 'Fraktion'] == 'keine':
#    df.loc[index, 'Fraktion'] = search_lost_party(zuordnung = zusatzdaten, id = row['Redner*in-ID'])

In [ ]:
print(df["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU                                                                     9382
SPD                                                                         8092
BÜNDNIS 90/DIE GRÜNEN                                                       4837
AfD                                                                         4825
FDP                                                                         4426
DIE LINKE                                                                   2810
Fraktion der Sozialdemokratischen Partei Deutschlands                       1195
Fraktion BÜNDNIS 90/DIE GRÜNEN                                              1042
Fraktion der Freien Demokratischen Partei                                    788
Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union     702
Die Linke                                                                    396
fraktionslos                                                                 303
Fraktionslos       

In [ ]:
#df[df['Fraktion']=='keine'].head(6)

In [ ]:
for i in range(0, len(df)):
    if df.iloc[i]["Redner*in Nachname"] == "Wissing":
        if (datetime.strptime(str(df.iloc[i]["Datum"]), "%d.%m.%Y") >= datetime.strptime("07.11.2024", "%d.%m.%Y")):
            df.loc[df.index[i], "Fraktion"] = "fraktionslos"

In [ ]:
 df[df["Redner*in Nachname"] == "Wissing"].head()


,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,Rede
494,20,19.12.2024,11003702,Volker,Wissing,Bundesminister für Digitales und Verkehr sowie...,fraktionslos,Frau Präsidentin! Liebe Kolleginnen und Kolleg...
2110,20,11.10.2024,11003702,Volker,Wissing,Bundesminister für Digitales und Verkehr,Fraktion der Freien Demokratischen Partei,Herzlichen Dank. – Frau Präsidentin! Liebe Kol...
2342,20,09.10.2024,11003702,Volker,Wissing,Bundesminister für Digitales und Verkehr,Fraktion der Freien Demokratischen Partei,Herzlichen Dank. – Frau Präsidentin! Liebe Kol...
2348,20,09.10.2024,11003702,Volker,Wissing,Bundesminister für Digitales und Verkehr,Fraktion der Freien Demokratischen Partei,"Ja, Herr Kollege, das Deutschlandticket ist ei..."
2350,20,09.10.2024,11003702,Volker,Wissing,Bundesminister für Digitales und Verkehr,Fraktion der Freien Demokratischen Partei,"Herr Kollege, der Bund hält sich ganz präzise ..."


In [ ]:
df.loc[df['Fraktion'] == 'Fraktion der Sozialdemokratischen Partei Deutschlands', 'Fraktion'] = 'SPD'

df.loc[df['Fraktion'] == 'Fraktion der Freien Demokratischen Partei', 'Fraktion'] = 'FDP'

df.loc[
    (df['Fraktion'] == 'Fraktion der Christlich Demokratischen Union/Christlich - Sozialen Union') |
    (df['Fraktion'] == 'CSU') |
     (df['Fraktion'] == 'CDU'),
    'Fraktion'
] = 'CDU/CSU'

df.loc[
    (df['Fraktion'] == 'Fraktion BÜNDNIS 90/DIE GRÜNEN') |
    (df['Fraktion'] == 'Bündnis90/DieGrünen'),
    'Fraktion'
] = 'BÜNDNIS 90/DIE GRÜNEN'

df.loc[
    (df['Fraktion'] == 'DieLinke') |
     (df['Fraktion'] == 'DIE LINKE.') |
    (df['Fraktion'] == 'Die Linke.'),
    'Fraktion'
] = 'DIELINKE'

In [ ]:
#df['Fraktion'].unique()
#print(df["Fraktion"].value_counts(), "\n\n\n")

In [ ]:
def clean_fraktion(fraktion):
  fraktion = fraktion.replace('\n', '')
  fraktion = fraktion.strip()
  fraktion = fraktion.replace(' ', '')
  fraktion = fraktion.lower()
  if ':' in fraktion:
    fraktion = fraktion.split(':')[0]
  return fraktion

In [ ]:
df['Fraktion'] = df['Fraktion'].apply(clean_fraktion)

In [ ]:
df = df.replace({'spd': 'SPD', 'cdu/csu':'CDU/CSU', 'bündnis90/diegrünen': 'Die Grünen', 'afd': 'AfD', 'dielinke': 'Die Linke', 'fdp': 'FDP'})
print(df["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU         10084
SPD              9287
Die Grünen       6221
FDP              5213
AfD              4825
Die Linke        3263
fraktionslos      536
bsw               152
keine               6
spdcdu/csu          6
Name: count, dtype: int64 





In [ ]:
df = df[df['Fraktion'].isin(['SPD', 'FDP', 'CDU/CSU','Die Grünen','AfD','Die Linke'])]

In [ ]:
df['Fraktion'].unique()

array(['SPD', 'CDU/CSU', 'Die Grünen', 'FDP', 'AfD', 'Die Linke'],
      dtype=object)

In [ ]:
print(df["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU       10084
SPD            9287
Die Grünen     6221
FDP            5213
AfD            4825
Die Linke      3263
Name: count, dtype: int64 





In [ ]:
df = df.drop_duplicates()
df.shape

(38891, 8)

In [ ]:
df.iloc[3]["Rede"]

'Sehr geehrte Frau Präsidentin! Liebe Kolleginnen und Kollegen! Der Kollege Lindner hat es gerade angesprochen: Wir werden hier heute im Rahmen der Abstimmung auch über das Ukrainepaket mit einem Volumen von 3 Milliarden Euro, ein Ukrainehilfspaket, entscheiden. Ich bin der Union dankbar, dass sie es mit uns auf den Weg bringt. Ich bin aber besonders den Sozialdemokraten und den Grünen dankbar, dass sie durch ihre Enthaltung dieses Paket möglich machen. Ich will ausdrücklich sagen, dass ich sehr dankbar dafür bin, weil es ein wichtiges Signal ist. Was aus meiner Sicht ein schwieriges Signal ist, ist die Haltung von Olaf Scholz dazu. Olaf Scholz war dieses 3-Milliarden-Euro-Paket einmal sehr, sehr wichtig. Im November letzten Jahres war es ihm so wichtig, dass er den damaligen Bundesfinanzminister entlassen hat. Er hatte ihm die Pistole auf die Brust gesetzt und wollte von ihm ultimativ, dass er eine Notlage erklärt, um dieses Ukrainepaket auf den Weg zu bringen. So wichtig war es Olaf 

In [ ]:
df.head()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,Rede
0,20,31.01.2025,11005010,Johannes,Arlt,keine,SPD,"Schönen guten Morgen, Frau Präsidentin! Liebe ..."
1,20,31.01.2025,11004924,Kerstin,Vieregge,keine,CDU/CSU,Sehr geehrte Frau Präsidentin! Sehr geehrter H...
2,20,31.01.2025,11004217,Tobias,Lindner,Staatsminister im Auswärtigen Amt,Die Grünen,Frau Präsidentin! Liebe Kolleginnen und Kolleg...
3,20,31.01.2025,11004828,Alexander,Müller,keine,FDP,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
4,20,31.01.2025,11005066,Hannes,Gnauck,keine,AfD,Frau Präsidentin! Meine Damen und Herren! Deut...


Die Reihenfolge der Beobachtungen ändern, sodass die späteste Beobachtung die früheste wird usw.

In [ ]:
df = df.iloc[::-1].reset_index(drop=True)

In [ ]:
df.head()

,Wahlperiode,Datum,Redner*in-ID,Redner*in Vorname,Redner*in Nachname,Rolle,Fraktion,Rede
0,19,15.01.2020,11004699,Astrid,Damerow,keine,CDU/CSU,Frau Präsidentin! Verehrte Kolleginnen und Kol...
1,19,15.01.2020,11004393,Johann,Saathoff,keine,SPD,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
2,19,15.01.2020,11003706,Artur,Auernhammer,keine,CDU/CSU,Verehrte Frau Präsidentin! Liebe Kolleginnen u...
3,19,15.01.2020,11003604,Friedrich,Ostendorff,keine,Die Grünen,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
4,19,15.01.2020,11003740,Heidrun,Bluhm-Förster,keine,Die Linke,Frau Präsidentin! Verehrte Kolleginnen und Kol...


In [ ]:
df.iloc[0]["Rede"]

'Frau Präsidentin! Verehrte Kolleginnen und Kollegen! Die Debatte hat es gezeigt: Niemand hier im Haus wird bezweifeln, dass Wasser eine Ressource von wirklich unschätzbarem Wert ist. Der Antrag der Grünen, der hier noch nicht so sehr besprochen wurde, hat die Überschrift: „Wasser muss sauber und bezahlbar bleiben“. Ich will an dieser Stelle nochmals wie übrigens bereits im Mai deutlich machen, dass wir das Ziel und das Anliegen der Antragsteller hier absolut teilen. Was wir allerdings nicht teilen, sind die Bewertungen und die Schlussfolgerungen, die Sie in Ihrem Antrag ziehen. Ich widerspreche hier nochmals ausdrücklich dem immer wieder erweckten Eindruck, dass wir bei der Wasserqualität in den vergangenen Jahren keine Fortschritte erzielt haben. Ganz im Gegenteil: Überall in Deutschland kann Wasser aus der Leitung getrunken werden. Das ist weiß Gott keine Selbstverständlichkeit. Verehrte Kolleginnen und Kollegen, Kernstück unseres Gewässerschutzes ist die europäische Wasserrahmenric

In [ ]:
print(df["Fraktion"].value_counts(), "\n\n\n")

Fraktion
CDU/CSU       10083
SPD            9287
Die Grünen     6221
FDP            5213
AfD            4824
Die Linke      3263
Name: count, dtype: int64 





In [ ]:
df.to_parquet(path + 'datatechlabs_2020_newclean.parquet')